In [1]:
# importando as bibliotecas necessárias

import pandas as pd
import numpy as np
from random import randint

# Biblioteca para abrir o arquivo
from zipfile import ZipFile

# Importando biblioteca de recomendação
from sklearn.neighbors import NearestNeighbors

# Arquivos estão aqui https://files.grouplens.org/datasets/movielens/ml-100k.zip

# Subindo os arquivos
files = ZipFile('ml-100k(1).zip')


In [2]:
#subindo os dataset e colocando em váriaves

df_filmes = pd.read_csv(files.open('ml-100k/u.item'), sep='|', encoding='latin-1', header=None)
df_nota = pd.read_csv(files.open('ml-100k/u.data'), sep='\t', header=None)

In [ ]:
#dataset de usuario não necessario

#df_usuarios = pd.read_csv(files.open('ml-100k/u.user'), sep='|', encoding='latin-1', header=None)
#df_usuarios.rename(columns={0 :'user_id', 1:'age', 2:'gender', 3:'occupation', 4:'zip_code'}, inplace=True)
#df_usuarios

In [3]:
#Renomeei e filtrei as colunas

df_nota.rename(columns={0 :'user_id', 1:'movie_id', 2:'rating', 3:'timestamp'}, inplace=True)
df_nota = df_nota[['user_id', 'movie_id', 'rating']]
df_nota

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [4]:
#Renomeei e filtrei as colunas

df_filmes.rename(columns={0: 'movie_id', 1: 'movie_title', 2: 'release_date', 3: 'video_release_date', 4: 'imdb_url', 5:
'unknown', 6: 'action', 7: 'adventure', 8: 'animation', 9: 'childrens', 10: 'comedy', 11: 'crime', 12:
'documentary', 13: 'drama', 14: 'fantasy', 15: 'film_noir', 16: 'horror', 17: 'musical', 18: 'mystery', 19:
'romance', 20: 'scifi', 21: 'thriller', 22: 'war', 23: 'western'}, inplace=True)
df_filmes = df_filmes[['movie_id','movie_title']]
df_filmes

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [5]:
# Juntando as tabelas. Agora mostra o usuário, o que ele viu e qual a nota ele deu

df = pd.merge(df_nota, df_filmes, on='movie_id', how='left')
df = df[['user_id', 'movie_id', 'movie_title', 'rating']]
df

,user_id,movie_id,movie_title,rating
0,196,242,Kolya (1996),3
1,186,302,L.A. Confidential (1997),3
2,22,377,Heavyweights (1994),1
3,244,51,Legends of the Fall (1994),2
4,166,346,Jackie Brown (1997),1
...,...,...,...,...
99995,880,476,"First Wives Club, The (1996)",3
99996,716,204,Back to the Future (1985),5
99997,276,1090,Sliver (1993),1
99998,13,225,101 Dalmatians (1996),2


In [ ]:
# Transformei em json para poder subir no firebase

df.to_json('filmes_notas.json')

In [6]:
# Subindo minhas notas para os filmes

contador = 0
nome = input('Digite seu nome: ')
while contador < 5:
    sorteio = randint(0, len(df_filmes) - 1)
    dica = df_filmes["movie_title"][sorteio]
    nota = int(input(f"Nota para {dica}"))
    if nota > 0 :
        contador += 1
        usuario_nome = {'user_id': nome, 'movie_title' : dica, "rating" : nota}
        df = df.append(usuario_nome, ignore_index=True)
print('FIM')

Digite seu nome: Alan
Nota para Exotica (1994)5
Nota para Bushwhacked (1995)5
Nota para Robert A. Heinlein's The Puppet Masters (1994)5
Nota para Sabrina (1954)5
Nota para Hunted, The (1995)5
FIM


In [7]:
# Pivot deixa em um formato que produz a intersecção do usuário e  filme, mostrando a nota

# Fillna preenche valores nulos. Importante na hora da fórmula

#não é possível usar apenas o pivot porque o indice é duplicado


recomendacao = df.pivot_table(index='user_id', columns='movie_title', values='rating').fillna(0)

# Criando o modelo e alimentando com as informações

modelo = NearestNeighbors(metric='cosine')
modelo.fit(recomendacao)

NearestNeighbors(metric='cosine')

In [8]:
# Executando o modelo de recomendação e dividindo em distância (indíce de proximidade) e vizinhos (os que são próximos )

distancia, vizinhos = modelo.kneighbors(recomendacao.loc[nome].values.reshape(1,-1), n_neighbors=6)

In [ ]:
vizinhos

In [ ]:
distancia

# Modelo 01 - Indicação do Vizinho Mais Próximo 

In [ ]:
# Mostrando os usuários parecidos e suas distâncias

for i in range(0, len(distancia.flatten())):
    if i == 0:
        print(f'Usuário: {nome}\n')
    else:
        print(f'Vizinho: {recomendacao.index[vizinhos.flatten()[i]]} \nDistância de: {distancia.flatten()[i]}')

In [ ]:
# Dataset do usuário com suas notas

ds_usuario = recomendacao.loc[nome].to_frame()
ds_usuario

In [ ]:
# Dataset do vizinho mais próximo com suas notas

vizinho_prox = recomendacao.index[vizinhos.flatten()[1]]
ds_vizinho = recomendacao.loc[vizinho_prox].to_frame()
ds_vizinho

In [ ]:
# Junção dos datasets e mostrando ordem decrescente

ds_recomendacao = pd.merge(ds_usuario, ds_vizinho, on='movie_title').sort_values(by=vizinho_prox, ascending=False)

# Filtrando com os que o usuário que receberá a informação não tenha visto

ds_recomendacao = ds_recomendacao.query(f'{nome} == 0 and {recomendacao.index[vizinhos.flatten()[i]]} > 0')

In [ ]:
ds_recomendacao[:5]

# Modelo 02 - Indicação dos cinco vizinhos mais próximos

In [9]:
# Dataset do usuário

ds_usuario = recomendacao.loc[nome].to_frame()
ds_usuario

,Alan
movie_title,
'Til There Was You (1997),0.0
1-900 (1994),0.0
101 Dalmatians (1996),0.0
12 Angry Men (1957),0.0
187 (1997),0.0
...,...
Young Guns II (1990),0.0
"Young Poisoner's Handbook, The (1995)",0.0
Zeus and Roxanne (1997),0.0


In [10]:
# Criando um novo dataset com o nome do filme e a nota dada pelo usuário, sem o indíce 
vizinhanca = ds_usuario
vizinhanca_df = vizinhanca.reset_index()
vizinhanca_df

,movie_title,Alan
0,'Til There Was You (1997),0.0
1,1-900 (1994),0.0
2,101 Dalmatians (1996),0.0
3,12 Angry Men (1957),0.0
4,187 (1997),0.0
...,...,...
1659,Young Guns II (1990),0.0
1660,"Young Poisoner's Handbook, The (1995)",0.0
1661,Zeus and Roxanne (1997),0.0
1662,unknown,0.0


In [11]:
# Acrescentando os vizinhos ao dataset 

for i in range(1, 6):
    vizinho_prox = recomendacao.index[vizinhos.flatten()[i]]
    ds_vizinho = recomendacao.loc[vizinho_prox].to_frame()
    vizinhanca = pd.merge(vizinhanca, ds_vizinho[vizinho_prox], on='movie_title')

In [15]:
#Resetando o index

vizinhanca = vizinhanca.reset_index()

In [16]:
# estabelecendo os vizinhos

pos1 = recomendacao.index[vizinhos.flatten()[1]]
pos2 = recomendacao.index[vizinhos.flatten()[2]]
pos3 = recomendacao.index[vizinhos.flatten()[3]]
pos4 = recomendacao.index[vizinhos.flatten()[4]]
pos5 = recomendacao.index[vizinhos.flatten()[5]]

In [17]:
# Acrescento os vizinhos e as notas dadas por eles. No final, faço uma coluna soma e indico os filmes favoritos do grupo

vizinhanca['SOMA'] = vizinhanca[pos1] + vizinhanca[pos2] + vizinhanca[pos3] + vizinhanca[pos4] + vizinhanca[pos5]
indicacao = vizinhanca.sort_values(by='SOMA', ascending=False)
indicacao = indicacao.query(f'{nome} == 0')
list(indicacao['movie_title'][:5])

['Apocalypse Now (1979)',
 'Silence of the Lambs, The (1991)',
 'Apollo 13 (1995)',
 'Toy Story (1995)',
 "Schindler's List (1993)"]

In [18]:
indicacao

,movie_title,Alan,536,122,84,767,41,SOMA
87,Apocalypse Now (1979),0.0,4.0,5.0,0.0,5.0,5.0,19.0
1329,"Silence of the Lambs, The (1991)",0.0,4.0,0.0,4.0,5.0,4.0,17.0
88,Apollo 13 (1995),0.0,5.0,4.0,0.0,4.0,4.0,17.0
1523,Toy Story (1995),0.0,5.0,0.0,2.0,5.0,4.0,16.0
1281,Schindler's List (1993),0.0,5.0,0.0,5.0,0.0,4.0,14.0
...,...,...,...,...,...,...,...,...
600,Ghosts of Mississippi (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0
599,Ghost in the Shell (Kokaku kidotai) (1995),0.0,0.0,0.0,0.0,0.0,0.0,0.0
595,Getting Even with Dad (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0
594,Getting Away With Murder (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Exportando

In [ ]:
#exportando o modelo 

import pickle
filename = 'modelo.pk1'
pickle.dump(modelo, open(filename, 'wb'))